In [19]:
import pandas as pd


In [20]:
df = pd.read_csv('output_train.csv')

In [21]:
unique_values = df['individualPosition'].unique()
print(unique_values)

['TOP' 'JUNGLE' 'MIDDLE' 'BOTTOM' 'UTILITY' 'Invalid']


In [22]:
df = df[df['individualPosition'] != 'Invalid']
most_common_champions = df['championName'].value_counts()

print(most_common_champions)

championName
Caitlyn    1946
Lux        1420
Jinx       1390
Jhin       1244
Teemo      1063
           ... 
Quinn        93
Kled         69
Rumble       68
Ivern        67
RekSai       48
Name: count, Length: 169, dtype: int64


In [23]:
unique_values = df['individualPosition'].unique()
print(unique_values)

['TOP' 'JUNGLE' 'MIDDLE' 'BOTTOM' 'UTILITY']


In [24]:

valid_positions = ['TOP', 'JUNGLE', 'MIDDLE', 'BOTTOM', 'UTILITY']
df = df[df['individualPosition'].isin(valid_positions)]

winners = df[df['win'] == True].groupby(['matchId', 'individualPosition'])['championName'].first().reset_index()
losers = df[df['win'] == False].groupby(['matchId', 'individualPosition'])['championName'].first().reset_index()

winners_pivot = winners.pivot(index='matchId', columns='individualPosition', values='championName')
winners_pivot.columns = [f'winner_{pos}' for pos in winners_pivot.columns]

losers_pivot = losers.pivot(index='matchId', columns='individualPosition', values='championName')
losers_pivot.columns = [f'loser_{pos}' for pos in losers_pivot.columns]

# Combine winners and losers into a single DataFrame
final_df = pd.concat([winners_pivot, losers_pivot], axis=1)

# Reset index for a clean DataFrame
final_df.reset_index(inplace=True)

# Display the resulting DataFrame
print(final_df)

              matchId winner_BOTTOM winner_JUNGLE winner_MIDDLE winner_TOP  \
0     EUW1_6452163746         Kaisa        Graves      Tristana   Renekton   
1     EUW1_6638193488          Jinx         Ivern          Ahri      Quinn   
2     EUW1_6648469461          Jinx        Rammus        Veigar     Illaoi   
3     EUW1_6648507818        Lucian         Briar         Yasuo      Nasus   
4     EUW1_6648528130          Jinx         Elise         Jayce       Sett   
...               ...           ...           ...           ...        ...   
6488  EUW1_7200446863   MissFortune          Kayn        Viktor      Garen   
6489  EUW1_7200455653        Ezreal            Vi       Morgana    Ambessa   
6490  EUW1_7200464488          Ashe       Shyvana       Leblanc   Pantheon   
6491  EUW1_7200473060       Caitlyn        Khazix       Ambessa      Nasus   
6492  EUW1_7200483587          Jinx       Evelynn         Talon      Urgot   

     winner_UTILITY loser_BOTTOM  loser_JUNGLE loser_MIDDLE los

In [25]:
final_df.to_csv('match_data.csv', index=False)

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report

df = final_df.copy()
df_loser = final_df.copy()

# Encode the target column (winner_TOP)
label_encoder = LabelEncoder()
df['winner_TOP_encoded'] = label_encoder.fit_transform(df['winner_TOP'])


# Encode the loser target column
label_encoder_anti = LabelEncoder()
df_loser['loser_TOP_encoded'] = label_encoder_anti.fit_transform(df_loser['loser_TOP'])



# Prepare the features and target
target = 'winner_TOP_encoded'
features = [col for col in df.columns if col not in ['matchId', 'winner_TOP', target]]

# Prepare features and target for antimodel
target_anti = 'loser_TOP_encoded'
features_anti = [col for col in df_loser.columns if col not in ['matchId', 'loser_TOP', target_anti]]




# Encode all categorical features
for col in features:
    if df[col].dtype == 'object':  # If the column is categorical
        df[col] = LabelEncoder().fit_transform(df[col])

X = df[features]
y = df[target]

# Encode all categorical features loser
for col in features_anti:
    if df_loser[col].dtype == 'object':  # If the column is categorical
        df_loser[col] = LabelEncoder().fit_transform(df_loser[col])

loser_X = df_loser[features_anti]
loser_y = df_loser[target_anti]

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the Logistic Regression model
model = LogisticRegression(random_state=42, max_iter=2000)
model.fit(X_train, y_train)

antimodel = LogisticRegression(random_state=42, max_iter=2000)
antimodel.fit(loser_X,loser_y)

# Evaluate the model
y_pred = model.predict(X_test)

# Get unique classes in y_test (actual test labels)
unique_classes = sorted(set(y_test))

# Update target_names to include only the present classes
target_names = []
for cls in unique_classes:
    if cls < len(label_encoder.classes_):
        # Safely map back to the original label
        target_names.append(label_encoder.inverse_transform([cls])[0])
    else:
        # Handle invalid class IDs gracefully
        target_names.append(f"Unknown_{cls}")

# Ensure all target_names are strings
target_names = [str(name) for name in target_names]

# Generate classification report
print(classification_report(y_test, y_pred, labels=unique_classes, target_names=target_names))





c:\Users\benha\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
def transform_perspective(input_dict):
    # Extract keys and values from the input
    transformed_dict = {}
    
    for key, value in input_dict.items():
        # Skip "loser_TOP" entry
        if key == 'loser_TOP':
            continue
        
        # Reverse winner to loser and vice versa
        if key.startswith('winner_'):
            new_key = key.replace('winner_', 'loser_')
        elif key.startswith('loser_'):
            new_key = key.replace('loser_', 'winner_')
        else:
            new_key = key  # Keep key unchanged if it doesn't match patterns
        
        transformed_dict[new_key] = value
    
    return transformed_dict

In [ ]:
def predict_winner_top(input_features, model, label_encoder, features):
    import numpy as np
    
    # Validate input features
    missing_features = [f for f in features if f not in input_features]
    if missing_features:
        raise ValueError(f"Missing required features: {missing_features}")

    # Create a DataFrame for the input
    input_df = pd.DataFrame([input_features])

    # Ensure categorical columns are encoded as the model expects
    for col in features:
        if input_df[col].dtype == 'object':  # Encode only if the column is categorical
            try:
                # Check if the label exists in the encoder's classes
                if input_df[col].iloc[0] not in label_encoder.classes_:
                    input_df[col] = 'Unknown'  # Handle unseen label by setting it as 'Unknown'
                input_df[col] = label_encoder.transform(input_df[col])
            except ValueError:
                # Handle unseen labels gracefully by assigning 'Unknown'
                input_df[col] = label_encoder.transform(['Unknown'])

    # Ensure the feature order matches the model's expectation
    input_df = input_df[features]

    # Make prediction
    probabilities = model.predict_proba(input_df)[0]
    prediction_encoded = np.argmax(probabilities)
    prediction_decoded = label_encoder.inverse_transform([prediction_encoded])[0]

   
    # Print probabilities for debugging
    print("\nProbabilities for each class (sorted highest to lowest):")
    sorted_probabilities = sorted(
        zip(label_encoder.classes_, probabilities),
        key=lambda x: x[1],
        reverse=True
    )
    for champion, prob in sorted_probabilities:
        print(f"{champion}: {prob:.4f}")


    return prediction_decoded, sorted_probabilities



In [ ]:
def predict_loser_top(input_features, model, label_encoder, features):
    import numpy as np
    
    # Validate input features
    missing_features = [f for f in features if f not in input_features]
    if missing_features:
        raise ValueError(f"Missing required features: {missing_features}")

    # Create a DataFrame for the input
    input_df = pd.DataFrame([input_features])

    # Ensure categorical columns are encoded as the model expects
    for col in features:
        if input_df[col].dtype == 'object':  # Encode only if the column is categorical
            try:
                # Check if the label exists in the encoder's classes
                if input_df[col].iloc[0] not in label_encoder.classes_:
                    input_df[col] = 'Unknown'  # Handle unseen label by setting it as 'Unknown'
                input_df[col] = label_encoder.transform(input_df[col])
            except ValueError:
                # Handle unseen labels gracefully by assigning 'Unknown'
                input_df[col] = label_encoder.transform(['Unknown'])

    # Ensure the feature order matches the model's expectation
    input_df = input_df[features]

    # Make prediction
    probabilities = model.predict_proba(input_df)[0]
    prediction_encoded = np.argmax(probabilities)
    prediction_decoded = label_encoder.inverse_transform([prediction_encoded])[0]

   
    # Print probabilities for debugging
    print("\nProbabilities for each class (sorted highest to lowest):")
    sorted_probabilities = sorted(
        zip(label_encoder.classes_, probabilities),
        key=lambda x: x[1],
        reverse=True
    )
    for champion, prob in sorted_probabilities:
        print(f"{champion}: {prob:.4f}")


    return prediction_decoded, sorted_probabilities

In [ ]:

example_input = {
    'winner_BOTTOM': 'Fiora',
    'winner_JUNGLE': 'Viego',
    'winner_MIDDLE': 'Garen',
    'winner_UTILITY': 'Fiora',
    'loser_BOTTOM': 'Fiora',
    'loser_JUNGLE': 'TwistedFate',
    'loser_MIDDLE': 'Fiora',
    'loser_TOP': 'MonkeyKing',
    'loser_UTILITY': 'Fiora'
}

# Predict the winner_TOP champion
try:
    prediction,probs_winner = predict_winner_top(
        input_features=example_input,
        model=model,
        label_encoder=label_encoder,
        features=features
    )
    print(f"Predicted winner_TOP champion: {prediction}")
except ValueError as e:
    print(f"Error: {e}")
    



Probabilities for each class (sorted highest to lowest):
Ambessa: 0.0519
MonkeyKing: 0.0479
Darius: 0.0451
Garen: 0.0419
XinZhao: 0.0388
Jax: 0.0370
Nami: 0.0299
TahmKench: 0.0270
Sejuani: 0.0253
Swain: 0.0252
DrMundo: 0.0242
Kennen: 0.0224
Kayle: 0.0212
Quinn: 0.0203
Viego: 0.0198
Renekton: 0.0195
Aatrox: 0.0189
Warwick: 0.0188
Shen: 0.0182
Chogath: 0.0174
Soraka: 0.0173
Illaoi: 0.0172
AurelionSol: 0.0165
Heimerdinger: 0.0161
Olaf: 0.0154
Irelia: 0.0154
Fiora: 0.0153
Malphite: 0.0150
KSante: 0.0145
Lillia: 0.0136
TwistedFate: 0.0134
Seraphine: 0.0133
Gnar: 0.0127
Gragas: 0.0112
Gwen: 0.0103
Gangplank: 0.0103
Shaco: 0.0102
Volibear: 0.0102
Taric: 0.0096
Jayce: 0.0095
Orianna: 0.0094
Camille: 0.0089
Yone: 0.0083
Viktor: 0.0082
Aurora: 0.0081
Skarner: 0.0081
Ryze: 0.0074
Briar: 0.0073
Sion: 0.0066
Riven: 0.0058
Tryndamere: 0.0056
Teemo: 0.0055
Velkoz: 0.0055
Vi: 0.0052
Akali: 0.0049
Kayn: 0.0046
MasterYi: 0.0046
Pantheon: 0.0043
Cassiopeia: 0.0036
Kled: 0.0035
Nunu: 0.0033
Ornn: 0.0027


In [ ]:
example_loser = transform_perspective(example_input)

# Predict the winner_TOP champion
try:
    prediction,probs_loser = predict_loser_top(
        input_features=example_input,
        model=antimodel,
        label_encoder=label_encoder,
        features=features_anti
    )
    print(f"Predicted loser_TOP champion: {prediction}")
    print(probs_loser)
except ValueError as e:
    print(f"Error: {e}")


Probabilities for each class (sorted highest to lowest):
Gwen: 0.0847
Vayne: 0.0660
Anivia: 0.0408
Nocturne: 0.0315
Vi: 0.0290
Ezreal: 0.0279
Leblanc: 0.0267
Orianna: 0.0250
Heimerdinger: 0.0244
nan: 0.0241
Jhin: 0.0238
Ekko: 0.0234
Nami: 0.0205
Volibear: 0.0202
DrMundo: 0.0182
Aatrox: 0.0181
Singed: 0.0180
TahmKench: 0.0171
Riven: 0.0164
Kaisa: 0.0160
Sylas: 0.0153
Kassadin: 0.0138
Shen: 0.0129
Hwei: 0.0126
Teemo: 0.0121
Vladimir: 0.0118
Kalista: 0.0110
Yone: 0.0101
Garen: 0.0094
Darius: 0.0086
Taric: 0.0084
Jax: 0.0079
Irelia: 0.0074
Tryndamere: 0.0074
Lillia: 0.0072
Nautilus: 0.0069
Rumble: 0.0068
Jinx: 0.0068
Rengar: 0.0064
Graves: 0.0062
Yuumi: 0.0062
Zoe: 0.0057
Sion: 0.0057
Blitzcrank: 0.0053
Akshan: 0.0051
Udyr: 0.0046
Yasuo: 0.0040
Urgot: 0.0038
Syndra: 0.0036
Sivir: 0.0035
Lulu: 0.0035
Zilean: 0.0033
Ryze: 0.0033
Viego: 0.0032
Ziggs: 0.0032
Senna: 0.0030
Malzahar: 0.0029
Gragas: 0.0029
Nilah: 0.0028
Yorick: 0.0028
Shyvana: 0.0028
Diana: 0.0028
Ornn: 0.0026
Akali: 0.0025
Zac:

In [ ]:
result = {champion: probs_winner.get(champion, 0) - probs_loser.get(champion, 0)
          for champion in probs_winner.keys() | probs_loser.keys()}

AttributeError: 'list' object has no attribute 'keys'